In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import soccerdata as sd
import hopsworks
from datetime import date

[12/21/25 21:18:13] INFO     No custom team name replacements found. You can configure these in       ]8;id=770849;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=394056;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\ASUS\soccerdata\config\teamname_replacements.json.                           

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=509766;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=93738;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\ASUS\soccerdata\config\league_dict.json.                                     

In [2]:
today = date.today()
today.month
if today.month < 8 :
    year = str(today.year-1)+ '-' + str(today.year)
else:
    year = str(today.year)+ '-' + str(today.year+1)

year

'2025-2026'

In [3]:
fbref = sd.FBref('ENG-Premier League', year)
# Fetch data
games = fbref.read_schedule()
games.to_csv(f'schedule/schedule_new_{year}.csv')

[12/21/25 21:18:36] INFO     Saving cached data to C:\Users\ASUS\soccerdata\data\FBref               ]8;id=234053;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=146316;file://c:\Users\ASUS\anaconda3\envs\fb\lib\site-packages\soccerdata\_common.py#263\263]8;;\

[12/21/25 21:18:37] WARNING  FutureWarning: The behavior of DataFrame concatenation with empty or   ]8;id=91161;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=619176;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             all-NA entries is deprecated. In a future version, this will no longer                
                             exclude empty or all-NA columns when determining the result dtypes. To                
                             retain the old behavior, exclude the relevant entries before the                      
                             concat operation.                                                                     
                                                                                                                   

In [10]:
sche_old = pd.read_csv(f'schedule/schedule_old_{year}.csv')
sche_old = sche_old.dropna(subset = ['match_report'])
sche_new = pd.read_csv(f'schedule/schedule_new_{year}.csv')
sche_new = sche_new.dropna(subset = ['match_report'])
if len(sche_new) > len(sche_old):
    print(1)
print(len(sche_old))
print(len(sche_new))
sche_new.iloc[len(sche_old):,:]

168
168


,league,season,game,week,day,date,time,home_team,home_xg,score,away_xg,away_team,attendance,venue,referee,match_report,notes,game_id


In [ ]:
if len(sche_new) > len(sche_old):
    options = Options()
    options.add_argument('--headless=new')

    # Make Chrome look more like a real browser
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
    root_url = 'https://fbref.com'
    games = sche_new.iloc[len(sche_old):,:]
    games['match_report'] = root_url + games['match_report']
    for url in games['match_report']:
        driver = webdriver.Chrome(options = options)
        if url != 'NaN':
            try:
                driver.get(url)
                time.sleep(3)
                
                # Get page title to identify teams
                page_title = driver.title
                # Usually: "Southampton vs Arsenal Match Summary | May 25, 2025"
                
                if 'vs.' in page_title:
                    home_team = page_title.split('vs.')[0].strip()
                    away_team = page_title.split('vs.')[1].split('Match')[0].strip()
                    date = ' '.join(page_title.split()[-5:-2])
                else:
                    home_team = "can't find vs1"
                    away_team = "can't find vs2"
                
                print(f"Home: {home_team}")
                print(f"Away: {away_team}")
                print(f"Time: {time}")
                # Find all summary tables
                tables = driver.find_elements(By.CSS_SELECTOR, "table[id*='stats_'][id*='summary']")
                
                # Process home team table (first summary table without '_a')
                home_tables = [t for t in tables if '_a' not in t.get_attribute('id')]
                if home_tables:
                    home_html = home_tables[0].get_attribute('outerHTML')
                    df_home = pd.read_html(home_html)[0]
                    
                    # Add Team column
                    df_home['Team'] = home_team
                    
                    # print(f"\n{home_team} Players with Team Column:")
                    # print(df_home[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head())
                    # df_home.to_csv(f'{home_team}_players_with_team_{date}.csv', index=False)
                
                # Process away team table (summary table with '_a')
                # away_tables = [t for t in tables if '_a' in t.get_attribute('id')]
                away_tables = home_tables
                if away_tables:
                    away_html = away_tables[-1].get_attribute('outerHTML')
                    df_away = pd.read_html(away_html)[0]
                    
                    # Add Team column
                    df_away['Team'] = away_team
                    
                    # print(f"\n{away_team} Players with Team Column:")
                    # print(df_away[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head())
                    # df_away.to_csv(f'{away_team}_players_with_team_{date}.csv', index=False)
                
                # Combine both teams
                if home_tables and away_tables:
                    combined_df = pd.concat([df_home, df_away], ignore_index=True)
                    # print(f"\nCombined DataFrame ({len(combined_df)} players):")
                    # print(combined_df[[('Unnamed: 0_level_0', 'Player'), ('Team', '')]].head(10))
                    combined_df.to_csv(f'source/used/{home_team}_{away_team}_{date}.csv', index=False)
            
            finally:
                driver.quit()

Home: Liverpool
Away: Bournemouth
Time: <module 'time' (built-in)>


[12/21/25 15:16:39] WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=529903;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=631262;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

                    WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=681453;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=735392;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

Home: Aston Villa
Away: Newcastle United
Time: <module 'time' (built-in)>


[12/21/25 15:16:49] WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=617889;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=291704;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

...

                    WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=747006;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=953890;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

Home: Wolverhampton Wanderers
Away: Brentford
Time: <module 'time' (built-in)>


[12/21/25 15:45:31] WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=337758;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=637846;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

                    WARNING  FutureWarning: Passing literal html to 'read_html' is deprecated and   ]8;id=654644;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py\warnings.py]8;;\:]8;id=663829;file://c:\Users\ASUS\anaconda3\envs\fb\lib\warnings.py#109\109]8;;\
                             will be removed in a future version. To read from a literal string,                   
                             wrap it in a 'StringIO' object.                                                       
                                                                                                                   

InvalidArgumentException: Message: invalid argument: missing command parameters
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff75e5488e5
	0x7ff75e548940
	0x7ff75e32165d
	0x7ff75e39c313
	0x7ff75e39b6f0
	0x7ff75e2eb325
	0x7ff75e860640
	0x7ff75e85af80
	0x7ff75e8796e6
	0x7ff75e565de4
	0x7ff75e56ed8c
	0x7ff75e2e9cb8
	0x7ff75e9bf008
	0x7ff95c0ce8d7
	0x7ff95c2ec53c


In [ ]:
games.to_csv(f'schedule/schedule_old_{year}.csv')

# Make Summary file

In [11]:
import os
root_dir = 'source/'
files = os.listdir(root_dir + 'used')
all_dfs = []
for file in files:
    if '.csv' in file:
        df = pd.read_csv(root_dir + 'used/' + file)
        df.iloc[0,-1] = 'Team'
        new_columns = [f"{col}_{df.iloc[0][col]}" for col in df.columns]
        df = df[1:]
        df.columns = new_columns
        all_dfs.append(df)

df = pd.concat(all_dfs,ignore_index = True)
df = df.dropna()
df.to_csv(root_dir + 'summary.csv', index = False)